In [1]:
from pathlib import Path

from ray import tune
from ray.air import Checkpoint, session

In [2]:
ray_results_path = Path("~/ray_results").expanduser()
assert ray_results_path.is_dir()

In [3]:
last_experiment = max(ray_results_path.glob('TrainLightGCN_*'), key=lambda x: x.stat().st_ctime)
print(f"Loading results from {last_experiment}...")
assert last_experiment.is_dir()

class TrainLightGCN:
    def __init__(self):
        raise NotImplementedError

# TODO: Create an experiment grid or whatever is called
tuner = tune.Tuner.restore(str(last_experiment), TrainLightGCN)
rg = tuner.get_results()
rg.get_best_result()

Loading results from /home/davo/ray_results/TrainLightGCN_2023-10-19_18-09-46...


2023-10-19 18:31:40,947	WARNING experiment_analysis.py:205 -- Failed to fetch metrics for 2 trial(s):
- TrainLightGCN_167299ac: FileNotFoundError('Could not fetch metrics for TrainLightGCN_167299ac: both result.json and progress.csv were not found at /home/davo/ray_results/TrainLightGCN_2023-10-19_18-09-46/TrainLightGCN_167299ac_5_trial_index=4,batch_size=6,conv_layers=2,embedding_dim=50,l2=0.0012,learning_rate=0.0877_2023-10-19_18-11-34')
- TrainLightGCN_c1c3d069: FileNotFoundError('Could not fetch metrics for TrainLightGCN_c1c3d069: both result.json and progress.csv were not found at /home/davo/ray_results/TrainLightGCN_2023-10-19_18-09-46/TrainLightGCN_c1c3d069_10_trial_index=4,batch_size=5,conv_layers=3,embedding_dim=90,l2=0.0002,learning_rate=0.0000_2023-10-19_18-22-30')


Result(
  metrics={'iteration': 15, 'loss': 0.42830423354392955, 'mf_loss': 0.13134396769086132, 'emb_loss': 0.2969602659522071, 'recall': 0.0003135093809087667, 'ndcg': 0.0005061691510128748, 'precision': 0.0007434944237918217, 'map': 7.155300027451312e-05},
  path='/home/davo/ray_results/TrainLightGCN_2023-10-19_18-09-46/TrainLightGCN_b887b41c_1_trial_index=0,batch_size=6,conv_layers=2,embedding_dim=50,l2=0.0012,learning_rate=0.0877_2023-10-19_18-09-50',
  filesystem='local',
  checkpoint=Checkpoint(filesystem=local, path=/home/davo/ray_results/TrainLightGCN_2023-10-19_18-09-46/TrainLightGCN_b887b41c_1_trial_index=0,batch_size=6,conv_layers=2,embedding_dim=50,l2=0.0012,learning_rate=0.0877_2023-10-19_18-09-50/checkpoint_000000)
)

In [4]:
# Read dataframe and drop some unneeded columns
rdf = rg.get_dataframe('ncdg', 'max').drop(columns=['timestamp', 'node_ip', 'pid', 'hostname', 'time_since_restore', 'iterations_since_restore', 'logdir'])
rdf = rdf.drop(columns=['date'])
# rdf = rdf[rdf['done']]
print(rdf.columns)

for c in ['config/batch_size']: #, 'config/embedding_dim']:
    rdf[c] = 2**rdf[c]

rdf.sort_values('loss').head(5)

Index(['iteration', 'loss', 'mf_loss', 'emb_loss', 'recall', 'ndcg',
       'precision', 'map', 'done', 'training_iteration', 'trial_id',
       'time_this_iter_s', 'time_total_s', 'checkpoint_dir_name',
       'config/batch_size', 'config/embedding_dim', 'config/conv_layers',
       'config/learning_rate', 'config/l2', 'config/__trial_index__'],
      dtype='object')


,iteration,loss,mf_loss,emb_loss,recall,ndcg,precision,map,done,training_iteration,trial_id,time_this_iter_s,time_total_s,checkpoint_dir_name,config/batch_size,config/embedding_dim,config/conv_layers,config/learning_rate,config/l2,config/__trial_index__
0,15,0.428304,0.131344,0.296960,0.000314,0.000506,0.000743,0.000072,True,3,b887b41c,15.633864,43.634389,None,64,50,2,0.08772,0.001214,0
1,15,0.513906,0.144161,0.369744,0.000012,0.000252,0.000235,0.000006,True,3,4da34b60,31.033888,87.452540,None,64,50,2,0.08772,0.001214,1
2,15,0.578789,0.167185,0.411604,0.000285,0.000664,0.000551,0.000163,True,3,9f8694fc,56.320822,168.535029,None,64,50,2,0.08772,0.001214,2
3,15,0.615839,0.184260,0.431579,0.000156,0.001154,0.001187,0.000055,True,3,1631ec7e,156.874376,453.477913,None,64,50,2,0.08772,0.001214,3
5,15,0.672138,0.672070,0.000068,0.000000,0.000000,0.000000,0.000000,True,3,f00ee419,130.761698,354.418221,None,32,90,3,0.00001,0.000224,1
